In [17]:
# libs para processamento
import json

import geopandas as gpd
import pandas as pd

from hidromet.ANA import ANA
from hidromet.utils import dir_projeto
from shapely.geometry import Point
from hidromet import limpeza
from hidromet import modelos

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [2]:
diretorio = dir_projeto()

# Obter inventário de postos da ANA

In [3]:
ana = ANA()
inventario_plu = ana.inventario(tipoest=2)
inventario_plu.loc[:, ["latitude", "longitude"]] = inventario_plu.loc[:, ["latitude", "longitude"]].apply(pd.to_numeric)

# Recortar postos dentro do contorno das bacias

## Tratamento dos contornos

In [4]:
contornos = diretorio.joinpath("contornos", "contornos_smap.shp")

gdf = gpd.read_file(contornos)
crs = gdf.crs

In [5]:
coords = [Point(x) for x in zip(inventario_plu.longitude,
                                inventario_plu.latitude)]

postos_plu_ana = gpd.GeoDataFrame(inventario_plu, geometry = coords, crs = crs)

## Postos da ANA dentro dos contornos

In [6]:
bacias = list()
for ix in gdf.index:
    geometria = gdf.loc[ix, 'geometry']
    bacia = gdf.loc[ix, 'Nome']
    recorte_postos_ana = postos_plu_ana.geometry.within(geometria)
    postos_ana_dentro = postos_plu_ana.loc[recorte_postos_ana]
    postos_ana_dentro = postos_ana_dentro.assign(bacia=bacia)

    bacias.append(postos_ana_dentro)



In [7]:
df_postos_bacias = pd.concat(bacias)

In [8]:
dir_postos = diretorio.joinpath("arquivos", "postos_plu_ana_smap.csv")
df_postos_bacias.to_csv(dir_postos)

## Obter séries de chuva por bacia

In [11]:
dir_prec = diretorio.joinpath("arquivos", "series-chuva-ana")
dir_prec.mkdir(parents=True, exist_ok=True)

bacias_gp = df_postos_bacias.groupby("bacia")
bacias = bacias_gp.groups.keys()

info_bacias = list()
for bacia in bacias:
    arquivo_bacia = dir_prec.joinpath(f"{bacia}.csv")

    print(f"Obtendo séries de chuva para postos na bacia {bacia}.")
    bacia_df = bacias_gp.get_group(bacia)
    n_postos = len(bacia_df.index)

    chuvas_bacia = list()
    postos_sem_dado = list()
    postos_com_falha = list()
    postos_nao_representativos = list()
    postos_ok = list()
    for posto in bacia_df.index:
        df = ana.obter_chuva(cod_estacao=posto)
        df.index = pd.to_datetime(df.index)

        df = limpeza.remover_duplicados(df)
        df[posto] = limpeza.recortar_periodo(
            serie=df[posto], 
            inicio=pd.to_datetime("20000101"), 
            fim=pd.Timestamp('today')
            )
        df[posto] = limpeza.remover_outliers(df[posto], p=0.99)
        df[posto] = limpeza.substituir_negativos(df[posto])
        falhas = limpeza.contabilizar_falhas(df[posto])
        df = limpeza.remover_meses_nao_representativos(df)
        anos_disponiveis = limpeza.anos_disponiveis(df)
        n_dados = len(df)


        info_posto = modelos.Posto(
            latitude=bacia_df.loc[posto, 'latitude'],
            longitude=bacia_df.loc[posto, 'longitude'],
            codigo=posto,
            n_dados=n_dados
        )


        if not df.empty and falhas < 0.15 and anos_disponiveis > 8:
            postos_ok.append(info_posto)
            chuvas_bacia.append(df)
        elif df.empty:
            postos_sem_dado.append(info_posto)
        elif anos_disponiveis < 8: 
            postos_nao_representativos.append(info_posto)
        elif falhas > 0.15:
            postos_com_falha.append(info_posto)


    info_bacia = modelos.Bacia(
        serie_vazia=postos_sem_dado,
        nao_representativos=postos_nao_representativos,
        com_falhas=postos_com_falha,
        postos_ok=postos_ok,
        n_postos=n_postos,
        bacia=bacia
    )

    info_bacias.append(info_bacia)
    try:
        df_chuvas_bacia = pd.concat(chuvas_bacia, axis=1)
        df_chuvas_bacia.to_csv(arquivo_bacia)
    except (ValueError):
        print(
            f"Não há séries de precipitação disponíveis para os postos encontrados na bacia {bacia}"
        )

Obtendo séries de chuva para postos na bacia Barra Bonita.
Obtendo séries de chuva para postos na bacia Barra Bonita-Promissão.
Obtendo séries de chuva para postos na bacia Capivara.
Obtendo séries de chuva para postos na bacia Emborcação.
Não há séries de precipitação disponíveis para os postos encontrados na bacia Emborcação
Obtendo séries de chuva para postos na bacia Foz do Areia.
Obtendo séries de chuva para postos na bacia Furnas.
Obtendo séries de chuva para postos na bacia Iguaçu.
Obtendo séries de chuva para postos na bacia Itaipu.
Obtendo séries de chuva para postos na bacia Itá.
Obtendo séries de chuva para postos na bacia Promissão.
Obtendo séries de chuva para postos na bacia Rosana.
Obtendo séries de chuva para postos na bacia Serra da Mesa.
Obtendo séries de chuva para postos na bacia Sobradinho.
Obtendo séries de chuva para postos na bacia São Simão.
Obtendo séries de chuva para postos na bacia Tocantins.
Obtendo séries de chuva para postos na bacia Três Marias.
Obtendo

In [21]:
from pathlib import Path
for info in info_bacias:
    arquivo_json = Path(dir_prec, f'log_{info.bacia}.json')
    with open(arquivo_json, 'w') as f:
        json.dump(info.dict(), f)

In [32]:
from shapely.geometry import Point

info.bacia


'Água Vermelha'

In [33]:
postos_ok_bacia = info.postos_ok

In [34]:
for posto in postos_ok_bacia:
    print(posto)

latitude=-19.79 longitude=-49.6531 codigo='1949010'
latitude=-20.9517 longitude=-45.1531 codigo='2045035'
latitude=-20.7489 longitude=-46.7436 codigo='2046031'
latitude=-20.2497 longitude=-48.1561 codigo='2048103'
latitude=-21.6108 longitude=-45.49 codigo='2145046'
latitude=-21.2331 longitude=-45.1736 codigo='2145050'
latitude=-21.5861 longitude=-45.6631 codigo='2145057'
latitude=-21.7286 longitude=-46.6031 codigo='2146074'
latitude=-21.7217 longitude=-46.4556 codigo='2146075'
latitude=-21.5742 longitude=-46.8794 codigo='2146094'
latitude=-21.5675 longitude=-46.6236 codigo='2146117'
latitude=-21.6014 longitude=-46.9494 codigo='2146118'
latitude=-21.6253 longitude=-47.0094 codigo='2147167'
latitude=-21.9108 longitude=-47.3989 codigo='2147168'
latitude=-21.9603 longitude=-47.3253 codigo='2147169'
latitude=-21.1017 longitude=-47.7586 codigo='2147176'
latitude=-22.3794 longitude=-46.9003 codigo='2246190'
latitude=-22.3161 longitude=-46.8106 codigo='2246191'
latitude=-22.3858 longitude=-46.